In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import pandas as pd
import random
import wandb
from tqdm import tqdm
import os
import math

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.font_manager as fm

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
wandb.login(key= '1b5f670bdb4b8ed39a9bc34744dd738c9b33dede')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs24m022 (cs24m022-iit-madras-foundation) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Get Data

In [ ]:
# # My PC :- Used for initial experiments
# TRAIN_FilePath = 'dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv'
# DEV_FilePath = 'dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv'
# TEST_FilePath = 'dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv'

In [3]:
# # COLAB :- Used to check if code if gpu supporting and run experiments
# from google.colab import drive
# drive.mount('/content/drive')
# TRAIN_FilePath = '/content/drive/My Drive/hi/lexicons/hi.translit.sampled.train.tsv'
# DEV_FilePath = '/content/drive/My Drive/hi/lexicons/hi.translit.sampled.dev.tsv'
# TEST_FilePath = '/content/drive/My Drive/hi/lexicons/hi.translit.sampled.test.tsv'

Mounted at /content/drive


In [5]:
# KAGGLE :- Used for hyperparameter tuning
TRAIN_FilePath = "/kaggle/input/dakshina22/hi/lexicons/hi.translit.sampled.train.tsv"
DEV_FilePath = "/kaggle/input/dakshina22/hi/lexicons/hi.translit.sampled.dev.tsv"
TEST_FilePath = "/kaggle/input/dakshina22/hi/lexicons/hi.translit.sampled.test.tsv"

## Data Loading and Preprocessing

In [6]:
# Load train, dev and test datasets
train_df = pd.read_csv(TRAIN_FilePath, sep='\t', header=None)
dev_df = pd.read_csv(DEV_FilePath, sep='\t', header=None)
test_df = pd.read_csv(TEST_FilePath, sep='\t', header=None)


# Renaming the columns
train_df.columns = ['devanagari', 'latin', 'frequency']
dev_df.columns = ['devanagari', 'latin', 'frequency']
test_df.columns = ['devanagari', 'latin', 'frequency']

# Dataset Sizes
print(f"Train Dataset Size : {train_df.shape[0]}\nDev Dataset Size   : {dev_df.shape[0]}\nTest Dataset Size  : {test_df.shape[0]}")

Train Dataset Size : 44204
Dev Dataset Size   : 4358
Test Dataset Size  : 4502


In [7]:
train_df.head()

,devanagari,latin,frequency
0,अं,an,3
1,अंकगणित,ankganit,3
2,अंकल,uncle,4
3,अंकुर,ankur,4
4,अंकुरण,ankuran,3


In [8]:
class Vocabulary:
    def __init__(self):
        self.pad_token = "<pad>"
        self.sos_token = "<sos>"
        self.eos_token = "<eos>"
        self.unk_token = "<unk>"

        # Initialize mappings
        self.char2idx = {self.pad_token: 0, self.sos_token: 1, self.eos_token: 2, self.unk_token: 3}
        self.idx2char = {0: self.pad_token, 1: self.sos_token, 2: self.eos_token, 3: self.unk_token}
        self.vocab_size = 4

    def build_vocabulary(self, text_data):
        for text in text_data:
            text = str(text)
            for char in text:
                if char not in self.char2idx:
                    self.char2idx[char] = self.vocab_size
                    self.idx2char[self.vocab_size] = char
                    self.vocab_size += 1

    def encode(self, text, add_special_tokens=True):
        indices = []
        text = str(text)
        for char in text:
            indices.append(self.char2idx.get(char, self.char2idx[self.unk_token]))

        if add_special_tokens:
            indices = [self.char2idx[self.sos_token]] + indices + [self.char2idx[self.eos_token]]

        return indices

    def decode(self, indices, remove_special_tokens=True):
        chars = []
        keys = list(self.idx2char.keys())
        for idx in indices:
            if isinstance(idx, torch.Tensor):
                idx = idx.item()
            if idx in keys:
                char = self.idx2char[idx]
                if remove_special_tokens and char in [self.pad_token, self.sos_token, self.eos_token, self.unk_token]:
                    continue
                chars.append(char)

        return "".join(chars)


In [9]:
class TransliterationDataset(Dataset):
    def __init__(self, data_path, src_vocab, tgt_vocab):
        df = pd.read_csv(data_path, sep='\t', header=None)

        # Create Dataset
        self.source_sequences = []
        self.target_sequences = []

        for idx, row in df.iterrows():
            x_seq = src_vocab.encode(row[1])
            y_seq = tgt_vocab.encode(row[0])
            self.source_sequences.append(x_seq)
            self.target_sequences.append(y_seq)

    def __len__(self):
        return len(self.source_sequences)

    def __getitem__(self, idx):
        return torch.tensor(self.source_sequences[idx], dtype=torch.long), torch.tensor(self.target_sequences[idx], dtype=torch.long)


In [10]:
def collate_fn(batch):
    src_batch = [item[0] for item in batch]
    tgt_batch = [item[1] for item in batch]

    # Pad sequences
    src_batch_padded = pad_sequence(src_batch, batch_first=True, padding_value=0)
    tgt_batch_padded = pad_sequence(tgt_batch, batch_first=True, padding_value=0)

    return src_batch_padded, tgt_batch_padded

In [11]:
# Build Source and Target Vocabularies
src_vocab = Vocabulary()
tgt_vocab = Vocabulary()

train_df = pd.read_csv(TRAIN_FilePath, sep='\t', header=None)
src_text = []
tgt_text = []
for idx, row in train_df.iterrows():
    src_text.append(row[1])
    tgt_text.append(row[0])

src_vocab.build_vocabulary(src_text)
tgt_vocab.build_vocabulary(tgt_text)

In [12]:
# Train, Test and Dev (Validation) Dataset and Dataloaders
train_dataset = TransliterationDataset(TRAIN_FilePath, src_vocab, tgt_vocab)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

dev_dataset = TransliterationDataset(DEV_FilePath, src_vocab, tgt_vocab)
dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

test_dataset = TransliterationDataset(TEST_FilePath, src_vocab, tgt_vocab)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)


In [13]:
# Inspect if dataloader is created as desired
for batch in train_loader:
    inputs, targets = batch

    print("Inputs shape:", inputs.shape)
    print("Targets shape:", targets.shape)

    # Check one sample
    print("Sample input:", inputs[0])
    print("Sample target:", targets[0])
    print(src_vocab.decode(inputs[0]))
    print(tgt_vocab.decode(targets[0]))
    break

Inputs shape: torch.Size([32, 14])
Targets shape: torch.Size([32, 13])
Sample input: tensor([ 1,  5,  8, 17,  4,  4,  5,  2,  0,  0,  0,  0,  0,  0])
Sample target: tensor([ 1, 16,  9, 15, 21, 16,  2,  0,  0,  0,  0,  0,  0])
nidaan
निदान


## Vanilla Model

In [15]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers=1, cell_type="RNN", dropout=0.0):
        super().__init__()

        if num_layers == 1:
            dropout = 0

        self.cell_type = cell_type

        # Embedding Layer
        self.embedding = nn.Embedding(input_size, embedding_size)

        # Recurrent Layer
        if cell_type == "LSTM":
            self.recurrent_layer = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        elif cell_type == "GRU":
            self.recurrent_layer = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        else: # Default (RNN)
            self.recurrent_layer = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        # input: batch_size x seq_len

        embeddings = self.embedding(input) # (batch, seq_len, embedding_size)
        embeddings = self.dropout(embeddings)

        if self.cell_type == 'LSTM':
            outputs, (hidden, cell) = self.recurrent_layer(embeddings)
            return outputs, (hidden, cell)
        else:
            outputs, hidden = self.recurrent_layer(embeddings)
            return outputs, hidden


In [16]:
class Decoder(nn.Module):
    def __init__(self, output_size, embedding_size, hidden_size, num_layers=1, cell_type="RNN", dropout=0.0):
        super().__init__()

        if num_layers == 1:
            dropout = 0

        self.output_size = output_size
        self.cell_type = cell_type

        # Embedding Layer
        self.embedding = nn.Embedding(output_size, embedding_size)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        # Recurrent Layer
        if cell_type == "LSTM":
            self.recurrent_layer = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        elif cell_type == "GRU":
            self.recurrent_layer = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        else: # Default (RNN)
            self.recurrent_layer = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)

        # Output layer
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):

        input = input.unsqueeze(1)
        embeddings = self.embedding(input)
        embeddings = self.dropout(embeddings)

        if self.cell_type == "LSTM":
            hidden, cell = hidden
            outputs, (hidden, cell) = self.recurrent_layer(embeddings, (hidden, cell))
            hidden = (hidden, cell)
        else:
            outputs, hidden = self.recurrent_layer(embeddings, hidden)

        outputs = outputs.squeeze(1)
        prediction = self.fc_out(outputs)
        return prediction, hidden


In [17]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):

        batch_size = src.shape[0]
        tgt_len = tgt.shape[1]
        tgt_vocab_size = self.decoder.output_size

        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        if self.encoder.cell_type == 'LSTM':
            encoder_outputs, (hidden, cell) = self.encoder(src)
            decoder_hidden = (hidden, cell)
        else:
            encoder_outputs, hidden = self.encoder(src)
            decoder_hidden = hidden

        decoder_input = tgt[:, 0]
        for t in range(1, tgt_len):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            #print(decoder_output.shape)
            #break
            outputs[:, t] = decoder_output
            top = decoder_output.argmax(1)
            decoder_input = tgt[:, t] if random.random() < teacher_forcing_ratio else top
        return outputs


    def inference(self, src, max_len, sos_idx=1, eos_idx=2):

        batch_size = src.shape[0]
        tgt_len = max_len
        tgt_vocab_size = self.decoder.output_size
        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        if self.encoder.cell_type == 'LSTM':
            encoder_outputs, (hidden, cell) = self.encoder(src)
            decoder_hidden = (hidden, cell)
        else:
            encoder_outputs, hidden = self.encoder(src)
            decoder_hidden = hidden

        decoder_input = torch.tensor([sos_idx] * batch_size, device=self.device)
        for t in range(1, max_len):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[:, t] = decoder_output
            top = decoder_output.argmax(1)
            decoder_input = top

            # Check if all sequences have reached <eos>
            if (outputs == eos_idx).any(dim=1).all():
                break
        return outputs

## Training and Evaluation of Vanilla Model

In [18]:
def train(model, train_loader, optimizer, criterion, clip=1.0, teacher_forcing_ratio=0.5):
    model.train()
    epoch_loss = 0

    for i, (src, tgt) in enumerate(train_loader):
        src = src.to(device)
        tgt = tgt.to(device)
        #print("tgt", tgt)
        optimizer.zero_grad()

        output = model(src, tgt, teacher_forcing_ratio)
        #print("output", output)
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        tgt = tgt[:, 1:].reshape(-1)
        #print("tgt2", tgt)
        #print("output2", output)
        #break

        # Calculate loss
        loss = criterion(output, tgt)

        # Backpropagation
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Update parameters
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(train_loader)

In [19]:
def evaluate(model, val_loader, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, (src, tgt) in enumerate(val_loader):
            src = src.to(device)
            tgt = tgt.to(device)

            output = model.inference(src, tgt.shape[1])

            # Reshape output and target for loss calculation
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            tgt = tgt[:, 1:].reshape(-1)

            # Calculate loss
            loss = criterion(output, tgt)
            epoch_loss += loss.item()

    return epoch_loss / len(val_loader)

In [20]:
def transliterate(model, src_text, src_vocab, tgt_vocab, device, max_length=100):
    model.eval()

    # Convert source text to tensor
    src_indices = src_vocab.encode(src_text)
    src_tensor = torch.tensor(src_indices, dtype=torch.long).unsqueeze(0).to(device)

    # Get encoder outputs
    with torch.no_grad():
        if model.encoder.cell_type == 'LSTM':
            encoder_outputs, (hidden, cell) = model.encoder(src_tensor)
            decoder_hidden = (hidden, cell)
        else:
            encoder_outputs, hidden = model.encoder(src_tensor)
            decoder_hidden = hidden

    # Start with SOS token
    decoder_input = torch.tensor([tgt_vocab.char2idx[tgt_vocab.sos_token]], device=device)

    result_indices = [tgt_vocab.char2idx[tgt_vocab.sos_token]]

    for _ in range(max_length):
        with torch.no_grad():
            decoder_output, decoder_hidden = model.decoder(decoder_input, decoder_hidden)

        # Get the most likely next character
        top_token = decoder_output.argmax(1).item()
        result_indices.append(top_token)

        # Stop if EOS token
        if top_token == tgt_vocab.char2idx[tgt_vocab.eos_token]:
            break

        # Use predicted token as next input
        decoder_input = torch.tensor([top_token], device=device)

    # Convert indices to text
    result_text = tgt_vocab.decode(result_indices, remove_special_tokens=True)

    return result_text

In [21]:
# Accuracy calculation function
def calculate_accuracy(model, data_loader, src_vocab, tgt_vocab, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for src, tgt in data_loader:
            src = src.to(device)
            tgt = tgt.to(device)

            batch_size = src.shape[0]

            for i in range(batch_size):
                # Get source text and actual target text
                src_indices = src[i].tolist()
                src_text = src_vocab.decode(src_indices)
                actual_tgt_text = tgt_vocab.decode(tgt[i].tolist())

                # Get predicted transliteration
                predicted_tgt_text = transliterate(model, src_text, src_vocab, tgt_vocab, device)

                # Check if prediction matches
                if predicted_tgt_text == actual_tgt_text:
                    correct += 1
                total += 1

    return correct / total

## Sample Run of Vanilla Model

In [ ]:
# INPUT_SIZE = src_vocab.vocab_size
# OUTPUT_SIZE = tgt_vocab.vocab_size
# EMBEDDING_SIZE = 256
# HIDDEN_SIZE = 512
# NUM_LAYERS = 2
# CELL_TYPE = "LSTM"
# DROPOUT = 0.2
# LEARNING_RATE = 0.001
# NUM_EPOCHS = 10

# # Initialize encoder, decoder, and seq2seq model
# encoder = Encoder(
#     input_size=INPUT_SIZE,
#     embedding_size=EMBEDDING_SIZE,
#     hidden_size=HIDDEN_SIZE,
#     num_layers=NUM_LAYERS,
#     cell_type=CELL_TYPE,
#     dropout=DROPOUT
# )

# decoder = Decoder(
#     output_size=OUTPUT_SIZE,
#     embedding_size=EMBEDDING_SIZE,
#     hidden_size=HIDDEN_SIZE,
#     num_layers=NUM_LAYERS,
#     cell_type=CELL_TYPE,
#     dropout=DROPOUT
# )

# model = Seq2Seq(encoder, decoder, device).to(device)
# criterion = nn.CrossEntropyLoss(ignore_index=0)
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
# # Training loop
# print(f"Starting training for {NUM_EPOCHS} epochs...")

# for epoch in range(NUM_EPOCHS):
#     print(f"Epoch {epoch+1}/{NUM_EPOCHS}")

#     # Train model
#     train_loss = train(model, train_loader, optimizer, criterion)

#     # Evaluate model
#     valid_loss = evaluate(model, dev_loader, criterion)

#     print(f"Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f}")
#     print("="*20)

Starting training for 10 epochs...
Epoch 1/10
Train Loss: 1.6911 | Valid Loss: 1.2774
Epoch 2/10
Train Loss: 0.7447 | Valid Loss: 1.0841
Epoch 3/10
Train Loss: 0.5753 | Valid Loss: 1.0521
Epoch 4/10
Train Loss: 0.4678 | Valid Loss: 1.0716
Epoch 5/10
Train Loss: 0.3957 | Valid Loss: 1.0537
Epoch 6/10
Train Loss: 0.3445 | Valid Loss: 1.0727
Epoch 7/10
Train Loss: 0.3004 | Valid Loss: 1.1386
Epoch 8/10
Train Loss: 0.2758 | Valid Loss: 1.1567
Epoch 9/10
Train Loss: 0.2444 | Valid Loss: 1.2164
Epoch 10/10
Train Loss: 0.2251 | Valid Loss: 1.2348


In [ ]:
# val_accuracy = calculate_accuracy(model, dev_loader, src_vocab, tgt_vocab, device)
# print(f"\nValidation Accuracy: {val_accuracy:.4f}")


Validation Accuracy: 0.3502


## Hyperparameter Tuning of Vanilla Model

In [24]:
def sweep_hyperparameters(config=None):
    with wandb.init(config=config):
        config = wandb.config
        wandb.run.name = f"embedding_size_{str(config.embedding_size)}_num_layers_{str(config.num_layers)}_hidden_size_{config.hidden_size} \
                          _cell_type_{config.cell_type}_dp_{config.dropout}_lr_{config.learning_rate}_batch_{config.batch_size}"

        # Log in my details
        wandb.config.update({"NAME": "KILAPARTHI VISHNU VARDHAN", "ROLL NO.": "CS24M022"})

        train_dataset = TransliterationDataset(TRAIN_FilePath, src_vocab, tgt_vocab)
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, collate_fn=collate_fn)

        dev_dataset = TransliterationDataset(DEV_FilePath, src_vocab, tgt_vocab)
        dev_loader = DataLoader(dev_dataset, batch_size=config.batch_size, shuffle=False, collate_fn=collate_fn)

        INPUT_SIZE = src_vocab.vocab_size
        OUTPUT_SIZE = tgt_vocab.vocab_size

        EMBEDDING_SIZE = config.embedding_size
        HIDDEN_SIZE = config.hidden_size
        NUM_LAYERS = config.num_layers
        CELL_TYPE = config.cell_type
        DROPOUT = config.dropout
        LEARNING_RATE = config.learning_rate
        NUM_EPOCHS = 10

        # Initialize encoder, decoder, and seq2seq model
        encoder = Encoder(
            input_size=INPUT_SIZE,
            embedding_size=EMBEDDING_SIZE,
            hidden_size=HIDDEN_SIZE,
            num_layers=NUM_LAYERS,
            cell_type=CELL_TYPE,
            dropout=DROPOUT
        )

        decoder = Decoder(
            output_size=OUTPUT_SIZE,
            embedding_size=EMBEDDING_SIZE,
            hidden_size=HIDDEN_SIZE,
            num_layers=NUM_LAYERS,
            cell_type=CELL_TYPE,
            dropout=DROPOUT
        )

        model = Seq2Seq(encoder, decoder, device).to(device)
        criterion = nn.CrossEntropyLoss(ignore_index=0)
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

        for epoch in tqdm(range(NUM_EPOCHS)):
            # Train model
            train_loss = train(model, train_loader, optimizer, criterion)
            # Evaluate model
            valid_loss = evaluate(model, dev_loader, criterion)

            print(f"Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f}")
            # Log the evaluation metrics
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "validation_loss": valid_loss,
                })

        # Calculate accuracy on validation set
        val_accuracy = calculate_accuracy(model, dev_loader, src_vocab, tgt_vocab, device)
        print(f"\nValidation Accuracy: {val_accuracy:.4f}")
        wandb.log({
            "val_accuracy": val_accuracy
        })

In [25]:
sweep_config = {
    "method" : "bayes",
    "metric" : {"name": "val_accuracy", "goal": "maximize"},
    "parameters" : {
        "embedding_size" : {"values" : [16, 32, 64, 256]},
        "num_layers" : {"values" : [2, 3]},
        "hidden_size" : {"values" : [32, 128, 512]},
        "cell_type" : {"values" : ["RNN", "LSTM", "GRU"]},
        "dropout" : {"values" : [ 0.2, 0.3]},
        "learning_rate" : {"values" : [0.001, 0.0005]},
        "batch_size": {"values": [32, 64, 128]}
    }
}

sweep_id = wandb.sweep(sweep_config, project = "cs24m022_DA6401_Assignment3")

Create sweep with ID: rxo139kw
Sweep URL: https://wandb.ai/cs24m022-iit-madras-foundation/cs24m022_DA6401_Assignment3/sweeps/rxo139kw


In [26]:
wandb.agent('rxo139kw', function = sweep_hyperparameters, count = 30)

wandb: Agent Starting Run: ge8sbo9x with config:
wandb: 	batch_size: 64
wandb: 	cell_type: GRU
wandb: 	dropout: 0.3
wandb: 	embedding_size: 16
wandb: 	hidden_size: 512
wandb: 	learning_rate: 0.0005
wandb: 	num_layers: 2


 10%|█         | 1/10 [00:20<03:06, 20.71s/it]

Train Loss: 1.5995 | Valid Loss: 1.3383


 20%|██        | 2/10 [00:40<02:41, 20.17s/it]

Train Loss: 0.8974 | Valid Loss: 0.9699


...

Train Loss: 0.1461 | Valid Loss: 0.8616


100%|██████████| 10/10 [02:30<00:00, 15.07s/it]

Train Loss: 0.1365 | Valid Loss: 0.8908



Validation Accuracy: 0.3646


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▃▃▂▂▁▁▁▁▁
val_accuracy,▁
validation_loss,█▅▂▁▁▂▂▃▆█
epoch,9
train_loss,0.13648
val_accuracy,0.36462
validation_loss,0.89084


## Evaluate Best Vanilla Model on Test Dataset

In [27]:
INPUT_SIZE = src_vocab.vocab_size
OUTPUT_SIZE = tgt_vocab.vocab_size
EMBEDDING_SIZE = 256
HIDDEN_SIZE = 512
NUM_LAYERS = 3
CELL_TYPE = "GRU"
DROPOUT = 0.2
LEARNING_RATE = 0.0005
BATCH_SIZE = 64
NUM_EPOCHS = 10


train_dataset = TransliterationDataset(TRAIN_FilePath, src_vocab, tgt_vocab)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

dev_dataset = TransliterationDataset(DEV_FilePath, src_vocab, tgt_vocab)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

test_dataset = TransliterationDataset(TEST_FilePath, src_vocab, tgt_vocab)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# Initialize encoder, decoder, and seq2seq model
encoder = Encoder(
    input_size=INPUT_SIZE,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    cell_type=CELL_TYPE,
    dropout=DROPOUT
)

decoder = Decoder(
    output_size=OUTPUT_SIZE,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    cell_type=CELL_TYPE,
    dropout=DROPOUT
)

model = Seq2Seq(encoder, decoder, device).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in tqdm(range(NUM_EPOCHS)):
    # Train model
    train_loss = train(model, train_loader, optimizer, criterion)
    # Evaluate model
    valid_loss = evaluate(model, dev_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f}")

 10%|█         | 1/10 [00:24<03:38, 24.26s/it]

Train Loss: 1.5922 | Valid Loss: 1.1444


 20%|██        | 2/10 [00:48<03:15, 24.42s/it]

Train Loss: 0.7158 | Valid Loss: 1.0068


 30%|███       | 3/10 [01:13<02:51, 24.43s/it]

...

Train Loss: 0.2321 | Valid Loss: 1.1479


 90%|█████████ | 9/10 [03:45<00:25, 25.60s/it]

Train Loss: 0.2074 | Valid Loss: 1.1636


100%|██████████| 10/10 [04:11<00:00, 25.14s/it]

Train Loss: 0.1855 | Valid Loss: 1.2250


In [28]:
train_accuracy = calculate_accuracy(model, train_loader, src_vocab, tgt_vocab, device)
val_accuracy = calculate_accuracy(model, dev_loader, src_vocab, tgt_vocab, device)
test_accuracy = calculate_accuracy(model, test_loader, src_vocab, tgt_vocab, device)

print(f"Train Accuracy : {train_accuracy*100:6.2f}%")
print(f"Val Accuracy   : {val_accuracy*100:6.2f}%")
print(f"Test Accuracy  : {test_accuracy*100:6.2f}%")

Train Accuracy :  83.37%
Val Accuracy   :  38.96%
Test Accuracy  :  39.27%


In [15]:
model.eval()
latin = []
correct_native = []
predicted_native = []

with torch.no_grad():
    for src, tgt in test_loader:
        src = src.to(device)
        tgt = tgt.to(device)

        batch_size = src.shape[0]

        for i in range(batch_size):
            # Get source text and actual target text
            src_indices = src[i].tolist()
            src_text = src_vocab.decode(src_indices)
            actual_tgt_text = tgt_vocab.decode(tgt[i].tolist())

            # Get predicted transliteration
            predicted_tgt_text = transliterate(model, src_text, src_vocab, tgt_vocab, device)

            latin.append(src_text)
            correct_native.append(actual_tgt_text)
            predicted_native.append(predicted_tgt_text)

os.makedirs("predictions_vanilla", exist_ok=True)
df = pd.DataFrame({
    "Latin": latin,
    "Correct Native": correct_native,
    "Predicted Native": predicted_native
})
df["Correct"] = df["Correct Native"] == df["Predicted Native"]

excel_path = "predictions_vanilla/test_predictions.xlsx"
df.to_excel(excel_path, index=False)

wandb.init(project="cs24m022_da6401_assignment3", name="vanilla_seq2seq_run")

samples = df.sample(n=10)
wandb_table = wandb.Table(columns=["Latin", "Correct Native", "Predicted Native", "Correct"])

for _, row in samples.iterrows():
    wandb_table.add_data(row["Latin"], row["Correct Native"], row["Predicted Native"], str(row["Correct"]))

wandb.log({"Random Test Predictions Sample": wandb_table})


NameError: name 'model' is not defined

## SEQ2SEQ Model with Attention

In [14]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers=1, cell_type="RNN", dropout=0.0):
        super().__init__()

        if num_layers == 1:
            dropout = 0

        self.cell_type = cell_type

        # Embedding Layer
        self.embedding = nn.Embedding(input_size, embedding_size)

        # Recurrent Layer
        if cell_type == "LSTM":
            self.recurrent_layer = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        elif cell_type == "GRU":
            self.recurrent_layer = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        else: # Default (RNN)
            self.recurrent_layer = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        # input: batch_size x seq_len

        embeddings = self.embedding(input)
        embeddings = self.dropout(embeddings)

        if self.cell_type == 'LSTM':
            outputs, (hidden, cell) = self.recurrent_layer(embeddings)
            return outputs, (hidden, cell)
        else:
            outputs, hidden = self.recurrent_layer(embeddings)
            return outputs, hidden

In [15]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()

        # Attention parameters for: e_jt = Vatt^T tanh(Uatt * s_t-1 + Watt * c_j)
        self.Uatt = nn.Linear(hidden_size, hidden_size)
        self.Watt = nn.Linear(hidden_size, hidden_size)
        self.Vatt = nn.Linear(hidden_size, 1)

    def forward(self, hidden, encoder_outputs):

        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]

        if isinstance(hidden, tuple):  # LSTM
            hidden_state = hidden[0]
            hidden_state = hidden_state[-1]
        else:
            hidden_state = hidden[-1]

        hidden_expanded = hidden_state.unsqueeze(1)

        # Uatt * s_t-1
        uatt_term = self.Uatt(hidden_expanded)

        # Watt * c_j
        watt_term = self.Watt(encoder_outputs)


        combined = torch.tanh(uatt_term + watt_term)

        # Apply Vatt^T
        energy = self.Vatt(combined)
        energy = energy.squeeze(2)

        attention_weights = F.softmax(energy, dim=1)

        # Create context vector
        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs)
        context = context.squeeze(1)

        return context, attention_weights

In [16]:
class AttentionDecoder(nn.Module):
    def __init__(self, output_size, embedding_size, hidden_size, attention, num_layers=1, cell_type="RNN", dropout=0.0):
        super().__init__()

        if num_layers == 1:
            dropout = 0

        self.output_size = output_size
        self.cell_type = cell_type
        self.attention = attention
        self.hidden_size = hidden_size

        # Embedding Layer
        self.embedding = nn.Embedding(output_size, embedding_size)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        if cell_type == "LSTM":
            self.recurrent_layer = nn.LSTM(embedding_size + hidden_size, hidden_size, num_layers,
                                         dropout=dropout, batch_first=True)
        elif cell_type == "GRU":
            self.recurrent_layer = nn.GRU(embedding_size + hidden_size, hidden_size, num_layers,
                                        dropout=dropout, batch_first=True)
        else: # Default (RNN)
            self.recurrent_layer = nn.RNN(embedding_size + hidden_size, hidden_size, num_layers,
                                        dropout=dropout, batch_first=True)

        # Output layer
        self.fc_out = nn.Linear(hidden_size * 2, output_size)

    def forward(self, input, hidden, encoder_outputs):

        input = input.unsqueeze(1)
        embeddings = self.embedding(input)
        embeddings = self.dropout(embeddings)

        # Calculate attention context vector
        context, attention_weights = self.attention(hidden, encoder_outputs)

        # Concatenate embeddings and context vector
        context = context.unsqueeze(1)
        rnn_input = torch.cat((embeddings, context), dim=2)

        if self.cell_type == "LSTM":
            outputs, (hidden, cell) = self.recurrent_layer(rnn_input, hidden)
            hidden_state = hidden
            hidden_tuple = (hidden, cell)
        else:
            outputs, hidden = self.recurrent_layer(rnn_input, hidden)
            hidden_state = hidden
            hidden_tuple = hidden

        if isinstance(hidden_state, tuple):  # LSTM
            last_hidden = hidden_state[0][-1]
        else:
            last_hidden = hidden_state[-1]

        last_hidden = last_hidden.squeeze(0) if last_hidden.dim() > 2 else last_hidden

        outputs = outputs.squeeze(1)
        context = context.squeeze(1)

        output_vector = torch.cat((outputs, context), dim=1)
        prediction = self.fc_out(output_vector)

        return prediction, hidden_tuple, attention_weights

In [17]:
class Seq2SeqWithAttention(nn.Module):
    def __init__(self, encoder, decoder, attention, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.attention = attention
        self.device = device

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        # src: [batch_size, src_len]
        # tgt: [batch_size, tgt_len]

        batch_size = src.shape[0]
        tgt_len = tgt.shape[1]
        tgt_vocab_size = self.decoder.output_size

        # Tensor to store decoder outputs
        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        # Store attention weights for visualization
        attentions = torch.zeros(batch_size, tgt_len, src.shape[1]).to(self.device)

        # Encode the source sequence
        if self.encoder.cell_type == 'LSTM':
            encoder_outputs, (hidden, cell) = self.encoder(src)
            decoder_hidden = (hidden, cell)
        else:
            encoder_outputs, hidden = self.encoder(src)
            decoder_hidden = hidden

        # First input to the decoder is the <sos> token
        decoder_input = tgt[:, 0]

        # Start decoding
        for t in range(1, tgt_len):
            # Decode
            decoder_output, decoder_hidden, attention_weights = self.decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )

            # Store decoder output and attention
            outputs[:, t] = decoder_output
            attentions[:, t] = attention_weights

            # Teacher forcing: use ground truth or predicted token as next input
            teacher_force = random.random() < teacher_forcing_ratio
            top = decoder_output.argmax(1)
            decoder_input = tgt[:, t] if teacher_force else top

        return outputs, attentions

    def inference(self, src, max_len, sos_idx=1, eos_idx=2):
        # src: [batch_size, src_len]
        batch_size = src.shape[0]

        # Tensor to store decoder outputs
        outputs = torch.zeros(batch_size, max_len, self.decoder.output_size).to(self.device)

        # Store attention weights for visualization
        attentions = torch.zeros(batch_size, max_len, src.shape[1]).to(self.device)

        # Encode the source sequence
        if self.encoder.cell_type == 'LSTM':
            encoder_outputs, (hidden, cell) = self.encoder(src)
            decoder_hidden = (hidden, cell)
        else:
            encoder_outputs, hidden = self.encoder(src)
            decoder_hidden = hidden

        # First input to the decoder is the <sos> token
        decoder_input = torch.tensor([sos_idx] * batch_size, device=self.device)

        # Start decoding
        for t in range(1, max_len):
            # Decode
            decoder_output, decoder_hidden, attention_weights = self.decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )

            # Store decoder output and attention
            outputs[:, t] = decoder_output
            attentions[:, t] = attention_weights

            # Use predicted token as next input
            top = decoder_output.argmax(1)
            decoder_input = top

            # Check if all sequences have reached <eos>
            if (top == eos_idx).all():
                break

        return outputs, attentions


## Training and Evaluation of Attention Seq2Seq Model

In [18]:
def train_attention_model(model, train_loader, optimizer, criterion, clip=1.0, teacher_forcing_ratio=0.5):
    model.train()
    epoch_loss = 0

    for i, (src, tgt) in enumerate(train_loader):
        src = src.to(device)
        tgt = tgt.to(device)
        #print("tgt", tgt)
        optimizer.zero_grad()

        output, _ = model(src, tgt, teacher_forcing_ratio)
        #print("output", output)

        # Reshape output and target for loss calculation
        # output: [batch_size, tgt_len, output_dim]
        # tgt: [batch_size, tgt_len]
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)  # Remove first token (SOS)
        tgt = tgt[:, 1:].reshape(-1)  # Remove first token (SOS)
        #print("tgt2", tgt)
        #print("output2", output)
        #break
        # Calculate loss
        loss = criterion(output, tgt)

        # Backpropagation
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Update parameters
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(train_loader)


In [19]:
def evaluate_attention_model(model, val_loader, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, (src, tgt) in enumerate(val_loader):
            src = src.to(device)
            tgt = tgt.to(device)

            output, _ = model.inference(src, tgt.shape[1])

            # Reshape output and target for loss calculation
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            tgt = tgt[:, 1:].reshape(-1)

            # Calculate loss
            loss = criterion(output, tgt)
            epoch_loss += loss.item()

    return epoch_loss / len(val_loader)

In [20]:
def transliterate_attention_model(model, src_text, src_vocab, tgt_vocab, device, max_length=100):
    model.eval()

    # Convert source text to tensor
    src_indices = src_vocab.encode(src_text)
    src_tensor = torch.tensor(src_indices, dtype=torch.long).unsqueeze(0).to(device)

    # Get encoder outputs
    with torch.no_grad():
        if model.encoder.cell_type == 'LSTM':
            encoder_outputs, (hidden, cell) = model.encoder(src_tensor)
            decoder_hidden = (hidden, cell)
        else:
            encoder_outputs, hidden = model.encoder(src_tensor)
            decoder_hidden = hidden

    # Start with SOS token

    decoder_input = torch.tensor([tgt_vocab.char2idx[tgt_vocab.sos_token]], device=device)

    result_indices = [tgt_vocab.char2idx[tgt_vocab.sos_token]]

    for _ in range(max_length):
        with torch.no_grad():
            decoder_output, decoder_hidden, attention_weights = model.decoder(decoder_input, decoder_hidden, encoder_outputs)

        # Get the most likely next character
        top_token = decoder_output.argmax(1).item()
        result_indices.append(top_token)

        # Stop if EOS token
        if top_token == tgt_vocab.char2idx[tgt_vocab.eos_token]:
            break

        # Use predicted token as next input
        decoder_input = torch.tensor([top_token], device=device)

    # Convert indices to text
    result_text = tgt_vocab.decode(result_indices, remove_special_tokens=True)

    return result_text

In [21]:
# Accuracy calculation function
def calculate_accuracy_attention_model(model, data_loader, src_vocab, tgt_vocab, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for src, tgt in data_loader:
            src = src.to(device)
            tgt = tgt.to(device)

            batch_size = src.shape[0]

            for i in range(batch_size):
                # Get source text and actual target text
                src_indices = src[i].tolist()
                src_text = src_vocab.decode(src_indices)
                actual_tgt_text = tgt_vocab.decode(tgt[i].tolist())

                # Get predicted transliteration
                predicted_tgt_text = transliterate_attention_model(model, src_text, src_vocab, tgt_vocab, device)

                # Check if prediction matches
                if predicted_tgt_text == actual_tgt_text:
                    correct += 1
                total += 1

    return correct / total

## Sample Run of Attention Seq2Seq Model

In [20]:
# INPUT_SIZE = src_vocab.vocab_size
# OUTPUT_SIZE = tgt_vocab.vocab_size
# EMBEDDING_SIZE = 256
# HIDDEN_SIZE = 512
# NUM_LAYERS = 2
# CELL_TYPE = "LSTM"
# DROPOUT = 0.2
# LEARNING_RATE = 0.001
# NUM_EPOCHS = 10

# # Initialize encoder, attention, decoder, and seq2seqwithattention model
# encoder = Encoder(
#     input_size=INPUT_SIZE,
#     embedding_size=EMBEDDING_SIZE,
#     hidden_size=HIDDEN_SIZE,
#     num_layers=NUM_LAYERS,
#     cell_type=CELL_TYPE,
#     dropout=DROPOUT
# )

# attention = Attention(HIDDEN_SIZE)

# decoder = AttentionDecoder(
#     output_size=OUTPUT_SIZE,
#     embedding_size=EMBEDDING_SIZE,
#     hidden_size=HIDDEN_SIZE,
#     attention=attention,
#     num_layers=NUM_LAYERS,
#     cell_type=CELL_TYPE,
#     dropout=DROPOUT
# )

# model = Seq2SeqWithAttention(encoder, decoder, attention, device).to(device)
# criterion = nn.CrossEntropyLoss(ignore_index=0)
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [21]:
# # Training loop
# print(f"Starting training for {NUM_EPOCHS} epochs...")

# for epoch in range(NUM_EPOCHS):
#     print(f"Epoch {epoch+1}/{NUM_EPOCHS}")

#     # Train model
#     train_loss = train_attention_model(model, train_loader, optimizer, criterion)

#     # Evaluate model
#     valid_loss = evaluate_attention_model(model, dev_loader, criterion)

#     print(f"Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f}")
#     print("="*20)

Starting training for 10 epochs...
Epoch 1/10
Train Loss: 1.0431 | Valid Loss: 1.0556
Epoch 2/10
Train Loss: 0.6093 | Valid Loss: 0.9648
Epoch 3/10
Train Loss: 0.5000 | Valid Loss: 0.9998
Epoch 4/10
Train Loss: 0.4313 | Valid Loss: 0.9416
Epoch 5/10
Train Loss: 0.3761 | Valid Loss: 1.0126
Epoch 6/10
Train Loss: 0.3315 | Valid Loss: 0.9924
Epoch 7/10
Train Loss: 0.2904 | Valid Loss: 1.0180
Epoch 8/10
Train Loss: 0.2647 | Valid Loss: 1.0341
Epoch 9/10
Train Loss: 0.2411 | Valid Loss: 1.0818
Epoch 10/10
Train Loss: 0.2176 | Valid Loss: 1.1194


In [22]:
# val_accuracy = calculate_accuracy_attention_model(model, dev_loader, src_vocab, tgt_vocab, device)
# print(f"\nValidation Accuracy: {val_accuracy:.4f}")


Validation Accuracy: 0.3791


## Hyperparameter Sweep of Attention Seq2Seq Model

In [22]:
def sweep_hyperparameters(config=None):
    with wandb.init(config=config):
        config = wandb.config
        wandb.run.name = f"embedding_size_{str(config.embedding_size)}_num_layers_{str(config.num_layers)}_hidden_size_{config.hidden_size} \
                          _cell_type_{config.cell_type}_dp_{config.dropout}_lr_{config.learning_rate}_batch_{config.batch_size}"

        # Log in my details
        wandb.config.update({"NAME": "KILAPARTHI VISHNU VARDHAN", "ROLL NO.": "CS24M022"})

        train_dataset = TransliterationDataset(TRAIN_FilePath, src_vocab, tgt_vocab)
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, collate_fn=collate_fn)

        dev_dataset = TransliterationDataset(DEV_FilePath, src_vocab, tgt_vocab)
        dev_loader = DataLoader(dev_dataset, batch_size=config.batch_size, shuffle=False, collate_fn=collate_fn)

        INPUT_SIZE = src_vocab.vocab_size
        OUTPUT_SIZE = tgt_vocab.vocab_size

        EMBEDDING_SIZE = config.embedding_size
        HIDDEN_SIZE = config.hidden_size
        NUM_LAYERS = config.num_layers
        CELL_TYPE = config.cell_type
        DROPOUT = config.dropout
        LEARNING_RATE = config.learning_rate
        NUM_EPOCHS = 10

        # Initialize encoder, decoder, and seq2seq model
        encoder = Encoder(
            input_size=INPUT_SIZE,
            embedding_size=EMBEDDING_SIZE,
            hidden_size=HIDDEN_SIZE,
            num_layers=NUM_LAYERS,
            cell_type=CELL_TYPE,
            dropout=DROPOUT
        )

        attention = Attention(HIDDEN_SIZE)

        decoder = AttentionDecoder(
            output_size=OUTPUT_SIZE,
            embedding_size=EMBEDDING_SIZE,
            hidden_size=HIDDEN_SIZE,
            attention=attention,
            num_layers=NUM_LAYERS,
            cell_type=CELL_TYPE,
            dropout=DROPOUT
        )

        model = Seq2SeqWithAttention(encoder, decoder, attention, device).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        for epoch in tqdm(range(NUM_EPOCHS)):
            # Train model
            train_loss = train_attention_model(model, train_loader, optimizer, criterion)
            # Evaluate model
            valid_loss = evaluate_attention_model(model, dev_loader, criterion)

            print(f"Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f}")
            # Log the evaluation metrics
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "validation_loss": valid_loss,
                })

        # Calculate accuracy on validation set
        val_accuracy = calculate_accuracy_attention_model(model, dev_loader, src_vocab, tgt_vocab, device)
        print(f"\nValidation Accuracy: {val_accuracy:.4f}")
        wandb.log({
            "val_accuracy": val_accuracy
        })

In [23]:
sweep_config = {
    "method" : "bayes",
    "metric" : {"name": "val_accuracy", "goal": "maximize"},
    "parameters" : {
        "embedding_size" : {"values" : [16, 32, 64, 256]},
        "num_layers" : {"values" : [2, 3]},
        "hidden_size" : {"values" : [32, 128, 512]},
        "cell_type" : {"values" : ["RNN", "LSTM", "GRU"]},
        "dropout" : {"values" : [ 0.2, 0.3]},
        "learning_rate" : {"values" : [0.001, 0.0005]},
        "batch_size": {"values": [32, 64, 128]}
    }
}

# sweep_config = {
#     "method" : "bayes",
#     "metric" : {"name": "val_accuracy", "goal": "maximize"},
#     "parameters" : {
#         "embedding_size" : {"values" : [32]},
#         "num_layers" : {"values" : [1]},
#         "hidden_size" : {"values" : [512, 2048]},
#         "cell_type" : {"values" : [ "LSTM", "GRU"]},
#         "dropout" : {"values" : [ 0.4]},
#         "learning_rate" : {"values" : [ 0.0005]},
#         "batch_size": {"values": [32, 64]}
#     }
# }


sweep_id = wandb.sweep(sweep_config, project = "cs24m022_DA6401_Assignment3")

Create sweep with ID: tk2okbfr
Sweep URL: https://wandb.ai/cs24m022-iit-madras-foundation/cs24m022_DA6401_Assignment3/sweeps/tk2okbfr


In [24]:
sweep_id = 'ukpef1o2'
wandb.agent('ukpef1o2', function = sweep_hyperparameters, count = 40)

wandb: Agent Starting Run: nc08qgtb with config:
wandb: 	batch_size: 64
wandb: 	cell_type: RNN
wandb: 	dropout: 0.2
wandb: 	embedding_size: 256
wandb: 	hidden_size: 512
wandb: 	learning_rate: 0.0005
wandb: 	num_layers: 2
wandb: Ctrl + C detected. Stopping sweep.


## NOTE:- Sweep output for attention model is not presented as it was done in separate file on kaggle

## Evaluate Best Attention Seq2Seq Model on Test Dataset

In [25]:
# Best Hyperparameters
INPUT_SIZE = src_vocab.vocab_size
OUTPUT_SIZE = tgt_vocab.vocab_size
EMBEDDING_SIZE = 64
HIDDEN_SIZE = 512
NUM_LAYERS = 3
CELL_TYPE = "GRU"
DROPOUT = 0.4
LEARNING_RATE = 0.0005
BATCH_SIZE = 64
NUM_EPOCHS = 10

# Load train, dev and test dataloaders
train_dataset = TransliterationDataset(TRAIN_FilePath, src_vocab, tgt_vocab)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

dev_dataset = TransliterationDataset(DEV_FilePath, src_vocab, tgt_vocab)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

test_dataset = TransliterationDataset(TEST_FilePath, src_vocab, tgt_vocab)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)


# Initialize encoder, decoder, and seq2seq model
encoder = Encoder(
    input_size=INPUT_SIZE,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    cell_type=CELL_TYPE,
    dropout=DROPOUT
)

attention = Attention(HIDDEN_SIZE)

decoder = AttentionDecoder(
    output_size=OUTPUT_SIZE,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    attention=attention,
    num_layers=NUM_LAYERS,
    cell_type=CELL_TYPE,
    dropout=DROPOUT
)

model = Seq2SeqWithAttention(encoder, decoder, attention, device).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in tqdm(range(NUM_EPOCHS)):
    # Train model
    train_loss = train_attention_model(model, train_loader, optimizer, criterion)
    # Evaluate model
    valid_loss = evaluate_attention_model(model, dev_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f}")


 10%|█         | 1/10 [00:42<06:25, 42.81s/it]

Train Loss: 0.7711 | Valid Loss: 0.8770


 10%|█         | 1/10 [01:23<12:27, 83.07s/it]

Train Loss: 1.0691 | Valid Loss: 0.8941



 20%|██        | 2/10 [01:24<05:37, 42.24s/it]

...

Train Loss: 0.2420 | Valid Loss: 0.7353


 90%|█████████ | 9/10 [09:53<00:50, 50.66s/it]

Train Loss: 0.2267 | Valid Loss: 0.7429


100%|██████████| 10/10 [10:32<00:00, 63.22s/it]

Train Loss: 0.2143 | Valid Loss: 0.7443


In [26]:
train_accuracy = calculate_accuracy_attention_model(model, train_loader, src_vocab, tgt_vocab, device)
val_accuracy = calculate_accuracy_attention_model(model, dev_loader, src_vocab, tgt_vocab, device)
test_accuracy = calculate_accuracy_attention_model(model, test_loader, src_vocab, tgt_vocab, device)

print(f"Train Accuracy : {train_accuracy*100:6.2f}%")
print(f"Val Accuracy   : {val_accuracy*100:6.2f}%")
print(f"Test Accuracy  : {test_accuracy*100:6.2f}%")

KeyboardInterrupt: 

In [35]:
model.eval()
latin = []
correct_native = []
predicted_native = []

with torch.no_grad():
    for src, tgt in test_loader:
        src = src.to(device)
        tgt = tgt.to(device)

        batch_size = src.shape[0]

        for i in range(batch_size):
            # Get source text and actual target text
            src_indices = src[i].tolist()
            src_text = src_vocab.decode(src_indices)
            actual_tgt_text = tgt_vocab.decode(tgt[i].tolist())

            # Get predicted transliteration
            predicted_tgt_text = transliterate_attention_model(model, src_text, src_vocab, tgt_vocab, device)

            latin.append(src_text)
            correct_native.append(actual_tgt_text)
            predicted_native.append(predicted_tgt_text)

os.makedirs("predictions_attention", exist_ok=True)
df = pd.DataFrame({
    "Latin": latin,
    "Correct Native": correct_native,
    "Predicted Native": predicted_native
})
df["Correct"] = df["Correct Native"] == df["Predicted Native"]

excel_path = "predictions_attention/test_predictions.xlsx"
df.to_excel(excel_path, index=False)

wandb.init(project="cs24m022_da6401_assignment3", name="attention_seq2seq_run")

samples = df.sample(n=10)
wandb_table = wandb.Table(columns=["Latin", "Correct Native", "Predicted Native", "Correct"])

for _, row in samples.iterrows():
    wandb_table.add_data(row["Latin"], row["Correct Native"], row["Predicted Native"], str(row["Correct"]))

wandb.log({"Random Test Predictions Sample": wandb_table})


In [27]:
from google.colab import files
import requests

In [28]:
!mkdir -p ~/.fonts
font_url = "https://github.com/google/fonts/raw/main/ofl/notosansdevanagari/NotoSansDevanagari%5Bwdth%2Cwght%5D.ttf"
font_path = os.path.expanduser("~/.fonts/NotoSansDevanagari.ttf")

r = requests.get(font_url)
with open(font_path, 'wb') as f:
    f.write(r.content)

!rm -rf ~/.cache/matplotlib
fm.fontManager.addfont(font_path)
hindi_font = fm.FontProperties(fname=font_path)

In [29]:
def get_attention_weights(model, src_text, src_vocab, tgt_vocab, device, max_length=100):

    model.eval()
    src_indices = src_vocab.encode(src_text)
    src_tensor = torch.tensor(src_indices, dtype=torch.long).unsqueeze(0).to(device)

    # Get encoder outputs
    with torch.no_grad():
        if model.encoder.cell_type == 'LSTM':
            encoder_outputs, (hidden, cell) = model.encoder(src_tensor)
            decoder_hidden = (hidden, cell)
        else:
            encoder_outputs, hidden = model.encoder(src_tensor)
            decoder_hidden = hidden

    # Start with SOS token
    decoder_input = torch.tensor([tgt_vocab.char2idx[tgt_vocab.sos_token]], device=device)
    predicted = []
    attention_weights_list = []

    for _ in range(max_length):
        with torch.no_grad():
            decoder_output, decoder_hidden, attention_weights = model.decoder(decoder_input, decoder_hidden, encoder_outputs)

        attention_weights = attention_weights.squeeze(0).cpu().numpy()
        #print(len(src_indices), len(attention_weights))

        attention_weights_list.append(attention_weights)
        top_token = decoder_output.argmax(1).item()
        predicted.append(top_token)

        if top_token == tgt_vocab.char2idx[tgt_vocab.eos_token]:
            break

        decoder_input = torch.tensor([top_token], device=device)
    return attention_weights_list, predicted

In [30]:
def plot_attention_heatmap(model, dataloader, src_vocab, tgt_vocab, device, num_samples=10):

    model.eval()

    # Define plot
    num_rows = math.ceil(num_samples / 3)
    num_cols = min(num_samples, 3)

    plt.figure(figsize=(30, 10 * num_rows))

    # Get sample
    data = []
    for batch in dataloader:
        src, tgt = batch
        for i in range(len(src)):
            data.append((src[i], tgt[i]))
    samples = random.sample(data, min(len(data), num_samples))

    # Colormap
    colors = [(1, 1, 1), (0, 0, 1)]
    cmap = LinearSegmentedColormap.from_list("GBlue", colors, N=100)

    for idx, (src_tensor, tgt_tensor) in enumerate(samples):
        src_text = src_vocab.decode(src_tensor.tolist(), remove_special_tokens=True)
        tgt_text = tgt_vocab.decode(tgt_tensor.tolist(), remove_special_tokens=True)

        ax = plt.subplot(num_rows, num_cols, idx + 1)

        src_indices = src_vocab.encode(src_text)
        attention_weights_list, tgt_predictions_tokens = get_attention_weights(model, src_text, src_vocab, tgt_vocab, device)
        attention_mtx = np.array(attention_weights_list)

        src_chars = [src_vocab.idx2char[i] for i in src_indices]
        tgt_pred_chars = [tgt_vocab.idx2char[i] for i in tgt_predictions_tokens]

        im = ax.imshow(attention_mtx, cmap=cmap)

        # Set tick labels
        ax.set_xticks(np.arange(len(src_chars)))
        ax.set_yticks(np.arange(len(tgt_pred_chars)))
        ax.set_xticklabels(src_chars, fontproperties=hindi_font, fontsize=20)
        ax.set_yticklabels(tgt_pred_chars, fontproperties=hindi_font, fontsize=20)
        ax.set_title(f"Sample {idx+1}: '{src_text}' → '{tgt_text}' (pred: '{''.join(tgt_pred_chars)}')",fontproperties=hindi_font, fontsize=20)

    plt.tight_layout()
    plt.colorbar(im, ax=plt.gcf().get_axes())
    wandb.init(project="cs24m022_da6401_assignment3", name="Attention_Heatmap")
    wandb.log({"Attention Heat Maps": plt})
    plt.show()


In [ ]:
plot_attention_heatmap(model, test_loader, src_vocab, tgt_vocab, device, num_samples=10)

/tmp/ipykernel_35/3240096877.py:45: UserWarning: Glyph 8594 (\N{RIGHTWARDS ARROW}) missing from current font.
  plt.tight_layout()


/usr/local/lib/python3.11/dist-packages/wandb/sdk/data_types/image.py:307: UserWarning: Glyph 8594 (\N{RIGHTWARDS ARROW}) missing from current font.
  util.ensure_matplotlib_figure(data).savefig(buf, format=self.format)
/usr/local/lib/python3.11/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 8594 (\N{RIGHTWARDS ARROW}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)


In [41]:
# ───────────────────────────────────────────────────────────────
# Final Cell: Sample Words → Attention Viz → Log 3‑Column Table
# ───────────────────────────────────────────────────────────────
import os, json, random, torch, wandb
from IPython.display import HTML

# 1) Manual list of Latin words
sampled = ["Damn", "world", "need", "model", "example"]
print("Sampled words:", sampled)

# 2) Inline HTML generator
def create_interactive_connectivity(attn_matrix, input_seq, output_seq, filename="attention.html"):
    html_template = """<!DOCTYPE html><html><head><meta charset="UTF-8"><title>Attention</title><style>
 body{font-family:'Arial Unicode MS','Noto Sans Devanagari',sans-serif;margin:20px;text-align:center;}
 .container{display:inline-block;}
 .controls{margin:12px;}
 .slider{width:180px;}
 .output-chars,.input-chars{display:flex;justify-content:center;margin:10px;}
 .char{padding:6px 10px;margin:3px;font-size:18px;position:relative;cursor:pointer;min-width:24px;transition:all .2s;}
 .output-char{background:#f0f0f0;border-radius:4px;}
 .input-char{background:#e0e0e0;border-radius:4px;}
 .selected{background:#4caf50;color:#fff;font-weight:bold;box-shadow:0 0 6px rgba(76,175,80,.5);}
 .highlighted{background:rgba(76,175,80,.3);transform:scale(1.1);}
 .connection-line{position:absolute;background:rgba(0,200,0,.5);height:3px;transform-origin:left center;z-index:-1;pointer-events:none;}
</style></head><body>
 <div class="container">
  <div class="controls">
    Threshold:
    <input id="thr" type="range" min="0" max="100" value="30" class="slider">
    <span id="tv">0.30</span>
  </div>
  <div class="output-chars" id="outs"></div>
  <div class="input-chars" id="ins"></div>
 </div>
 <script>
  const A={attn_matrix},IN={input_seq},OUT={output_seq};
  let cur=0,thr=0.3;
  function init(){renderOut();renderIn();draw();
    document.getElementById('thr').oninput=e=>{thr=e.target.value/100;document.getElementById('tv').textContent=thr.toFixed(2);draw();};
    window.onresize=draw;
  }
  function renderOut(){let c=document.getElementById('outs');c.innerHTML='';
    OUT.forEach((ch,i)=>{let d=document.createElement('div');
      d.className=`char output-char ${i===cur?'selected':''}`;d.textContent=ch;d.dataset.i=i;
      d.onmouseover=d.onclick=()=>{cur=i;renderOut();draw();};c.appendChild(d);
    });
  }
  function renderIn(){let c=document.getElementById('ins');c.innerHTML='';
    IN.forEach((ch,i)=>{let d=document.createElement('div');
      d.className='char input-char';d.textContent=ch;d.dataset.i=i;c.appendChild(d);
    });
  }
  function draw(){
    document.querySelectorAll('.connection-line').forEach(e=>e.remove());
    document.querySelectorAll('.input-char').forEach(e=>e.classList.remove('highlighted'));
    let o=document.querySelector(`.output-char[data-i="${cur}"]`);if(!o)return;
    let R=o.getBoundingClientRect(),W=A[cur],M=Math.max(...W);
    W.forEach((w,i)=>{let n=w/M; if(n<thr) return;
      let inp=document.querySelector(`.input-char[data-i="${i}"]`);
      inp.classList.add('highlighted');let S=inp.getBoundingClientRect();
      let x1=R.left+R.width/2-window.scrollX,y1=R.top+R.height-window.scrollY,
          x2=S.left+S.width/2-window.scrollX,y2=S.top-window.scrollY;
      let L=Math.hypot(x2-x1,y2-y1),ang=Math.atan2(y2-y1,x2-x1)*180/Math.PI;
      let line=document.createElement('div');line.className='connection-line';
      Object.assign(line.style,{width:`${L}px`,left:`${x1}px`,top:`${y1}px`,transform:`rotate(${ang}deg)`,opacity:n});
      document.body.appendChild(line);
    });
  }
  document.addEventListener('DOMContentLoaded',init);
</script></body></html>"""

    # sanitize attention matrix
    clean = []
    for row in attn_matrix:
        if hasattr(row, "cpu"):
            row = row.cpu().numpy()
        clean.append([float(x) for x in row])
    attn_json = json.dumps(clean)
    inp_json  = json.dumps(input_seq)
    out_json  = json.dumps(output_seq)

    html = html_template.replace("{attn_matrix}", attn_json)\
                        .replace("{input_seq}", inp_json)\
                        .replace("{output_seq}", out_json)
    with open(filename, "w", encoding="utf-8") as f:
        f.write(html)
    return filename

# 3) Build & log the W&B table
wandb.init(project="cs24m022_da6401_assignment3", resume="allow")
table = wandb.Table(columns=["Input (Latin)", "Prediction (Devanagari)", "Attention (HTML)"])

model.eval()
for i, latin in enumerate(sampled):
    attn_weights, pred_ids = get_attention_weights(model, latin, src_vocab, tgt_vocab, device)
    pred_str = tgt_vocab.decode(pred_ids, remove_special_tokens=True)

    html_file = create_interactive_connectivity(
        attn_matrix=attn_weights,
        input_seq=list(latin),
        output_seq=list(pred_str),
        filename=f"attn_{i}.html"
    )
    html_txt = open(html_file, "r", encoding="utf-8").read()
    table.add_data(latin, pred_str, wandb.Html(html_txt))
    os.remove(html_file)

# Log & summary
wandb.log({"Interactive_Attention_Table": table})
wandb.run.summary["Interactive_Attention_Examples"] = table

# Optional display
display(HTML(html_file))


Sampled words: ['Damn', 'world', 'need', 'model', 'example']


epoch,9
train_loss,0.23786
val_accuracy,0.33869
validation_loss,0.74025
